# Covid-19 tracker - Cases, Mortality and Vaccinations

This small prototype project is to build a webscraper capable of gathering info on the current covid-19 pandemic, and map hotzones within countries. I'll potentially be making a nice GUI for this if it works well

In [3]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

## Webscraping for covid-19 data using beautifulsoup.
